In [10]:
%load_ext autoreload
%autoreload 2

In [11]:
import numpy as np
import torch 
import os 
import pandas as pd
from torch.utils.data import DataLoader

In [22]:
import detection

In [18]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [19]:
train_imgs_path = 'public_tests/splitted/train/images'
test_imgs_path = 'public_tests/splitted/test/images'
train_gt_path = 'public_tests/splitted/train/gt.csv'
test_gt_path = 'public_tests/splitted/test/gt.csv'

In [23]:
val_dataset = detection.FacesPointsDataset(test_imgs_path, test_gt_path, 'val')
val_loader = DataLoader(val_dataset, batch_size = 32, drop_last=True)

In [24]:
model = detection.train_detector(train_gt=train_gt_path, train_img_dir=train_imgs_path, fast_train=False, val_loader=val_loader)

d:\venvs\CV\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                | Params
----------------------------------------------
0 | model | FacesPointsDetector | 1.7 M 
----------------------------------------------
1.7 M     Trainable params
0         Non-trainable params
1.7 M     Total params
6.773     Total estimated model params size (MB)


Adjusting learning rate of group 0 to 5.0000e-04.
                                                                            

d:\venvs\CV\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
d:\venvs\CV\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 31: 100%|██████████| 150/150 [01:14<00:00,  2.01it/s, v_num=1, loss_step=13.30, val_loss_step=24.50, val_loss_epoch=15.40, loss_epoch=15.80]Epoch 31, Train_loss: 15.768


`Trainer.fit` stopped: `max_epochs=32` reached.


Epoch 31: 100%|██████████| 150/150 [01:15<00:00,  1.99it/s, v_num=1, loss_step=13.30, val_loss_step=24.50, val_loss_epoch=15.40, loss_epoch=15.80]


In [ ]:
# chkpt_path ='lightning_logs/version_18/checkpoints/epoch=15-step=4800.ckpt' #lightning_logs/version_17/checkpoints/epoch=15-step=4800.ckpt'

In [ ]:
def scale(current_shape, labels):
    x_scale = 100/current_shape[2]
    y_scale = 100/current_shape[1]
    transformed = labels.clone()
    for i in range(len(labels)):
        if i % 2 == 0:
            scale = x_scale
        else:
            scale = y_scale
        transformed[i] = int(transformed[i]*scale)
    return transformed

In [ ]:
def get_test_err(test_imgs_dir, test_gt_path, model):
    err = 0.0
    dataset = detection.FacesPointsDataset(test_imgs_dir, test_gt_path, mode='test', transform=None)
    for i in range(len(dataset)):
        img, label = dataset[i]
        pred = model(img[None,:]).detach()[0]
        # result[dataset.items[i][0]] = pred
        diff = ((pred - label)**2).mean()
        err += diff
    err /= len(dataset)
    return err

In [ ]:
import detection

In [ ]:
get_test_err(test_imgs_path, test_gt_path, model)

tensor(23.5695)